<a href="https://colab.research.google.com/github/Tyton25/tools/blob/main/Expenses_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd
import numpy as np
import re

## Functions

In [143]:
def refactor_debit_cols(my_df):
  columns = list(my_df.columns)
  columns = [c.lower() for c in columns]
  columns = [c.replace(' ', '_') for c in columns]
  my_df.columns = columns
  my_df.drop(columns='status', inplace=True)
  debit_df["date"] = pd.to_datetime(debit_df["date"])

  return my_df

In [144]:
def clean_orig_desc_col(raw_df):
  phone_pattern = r'\d{3}-\d{3}-\d{4}'
  my_df = raw_df[["original_description", "amount"]]
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('*')[0])
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('#')[0])
  my_df['original_description'] = my_df['original_description'].map(lambda x: re.split(phone_pattern, x)[0])
  my_df['original_description'] = my_df['original_description'].map(lambda x: re.split(r'dd ', x)[0])

  return my_df

In [145]:
def sum_orig_desc_col(raw_df):
  my_df = raw_df.groupby('original_description').agg('sum')
  my_df.sort_values(by=['original_description'], inplace=True)
  my_df.reset_index(inplace=True)

  return my_df

In [ ]:
def sum_category_col(raw_df):
  cat_df = debit_df[["category", "amount"]]
  cat_df = cat_df.groupby('category').agg("sum")
  cat_df.reset_index(inplace=True)

  return cat_df

In [ ]:
def sum_desc_col(raw_df):
  my_df = raw_df[["description", "amount"]]
  my_df = my_df.groupby('description').agg('sum')
  my_df.sort_values(by=['description'], inplace=True)
  my_df.reset_index(inplace=True)

  return my_df

In [146]:
debit_df = pd.read_csv("/content/debit_transactions_Jan-Aug_2023.csv")
debit_df

,Date,Description,Original Description,Category,Amount,Status
0,2023-08-10,CARIBOU COFFEE CO #1 081023,CARIBOU COFFEE CO #1 081023,Category Pending,-4.66,Pending
1,2023-08-10,KRISPY KREME #4110 081023,KRISPY KREME #4110 081023,Category Pending,-89.78,Pending
2,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35,Pending
3,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35,Pending
4,2023-08-10,PBC- DENVER CO3834 081023,PBC- DENVER CO3834 081023,Category Pending,-2.10,Pending
...,...,...,...,...,...,...
626,2023-01-03,Vasa Fitness,VASA FITNESS 800-748-4949 UT,Gym,-21.99,Posted
627,2023-01-03,Rokfin,ROKFIN 512-3876531 TX,Entertainment,-14.99,Posted
628,2023-01-03,YouTube Premium,GOOGLE *YouTubePremium G.CO/HELPPAY#CA,Entertainment,-12.97,Posted
629,2023-01-03,7-Eleven,SEI 38135 BROOMFIELD CO,Gas,-7.32,Posted


In [147]:
debit_df = refactor_debit_cols(debit_df)
debit_df

,date,description,original_description,category,amount
0,2023-08-10,CARIBOU COFFEE CO #1 081023,CARIBOU COFFEE CO #1 081023,Category Pending,-4.66
1,2023-08-10,KRISPY KREME #4110 081023,KRISPY KREME #4110 081023,Category Pending,-89.78
2,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35
3,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35
4,2023-08-10,PBC- DENVER CO3834 081023,PBC- DENVER CO3834 081023,Category Pending,-2.10
...,...,...,...,...,...
626,2023-01-03,Vasa Fitness,VASA FITNESS 800-748-4949 UT,Gym,-21.99
627,2023-01-03,Rokfin,ROKFIN 512-3876531 TX,Entertainment,-14.99
628,2023-01-03,YouTube Premium,GOOGLE *YouTubePremium G.CO/HELPPAY#CA,Entertainment,-12.97
629,2023-01-03,7-Eleven,SEI 38135 BROOMFIELD CO,Gas,-7.32


In [148]:
debit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  631 non-null    datetime64[ns]
 1   description           631 non-null    object        
 2   original_description  631 non-null    object        
 3   category              631 non-null    object        
 4   amount                631 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 24.8+ KB


In [80]:
debit_df["amount"].sum().round(2)

-6728.87

In [81]:
debit_df.category.unique()

array(['Category Pending', 'Fast Food', 'Utilities',
       'Electronics & Software', 'Service & Parts', 'Food & Dining',
       'Gas', 'Coffee Shops', 'Cash', 'Restaurants',
       'Newspapers & Magazines', 'Atm Fee', 'Interest Income',
       'Mortgage Payment', 'Hosting', 'Paycheck', 'Mobile Phone',
       'Movies & Dvds', 'Doctor', 'Music', 'Eyecare', 'Personal Care',
       'Auto & Transport', 'Television', 'Shopping', 'Entertainment',
       'Financial', 'Credit Card Payment', 'Sporting Goods',
       'Gifts & Donations', 'Health & Fitness', 'Education', 'Check',
       'Clothing', 'Business Services', 'Uncategorized', 'Bonus',
       'Home Improvement', 'Dentist', 'Transfer', 'Groceries', 'Books',
       'State Tax', 'Federal Tax', 'Laundry', 'Taxes', 'Parking', 'Gym'],
      dtype=object)

In [171]:
category_totals_df = sum_category_col(debit_df)
category_totals_df

,category,amount
0,Atm Fee,20.95
1,Auto & Transport,-289.32
2,Bonus,998.70
3,Books,-147.55
4,Business Services,-47.00
5,Cash,-1230.95
6,Category Pending,-7226.75
7,Check,-3382.13
8,Clothing,-2559.32
9,Coffee Shops,-258.31


In [169]:
debit_description_totals_df = sum_desc_col(debit_df)
debit_description_totals_df

,description,amount
0,7-Eleven,-1254.98
1,AAA Membership,-185.00
2,AMC Theatres,-39.39
3,ATM Fee Rebate,20.95
4,Adobe,-30.79
...,...,...
133,Xbox,-72.89
134,Xcel Energy,-1723.69
135,YouTube Premium,-90.79
136,Zappos.com,-122.86


In [149]:
debit_orig_description_amounts_df = clean_orig_desc_col(debit_df)
debit_orig_description_amounts_df[:50]

<ipython-input-144-a64b9d33a88e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('*')[0])
<ipython-input-144-a64b9d33a88e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('#')[0])
<ipython-input-144-a64b9d33a88e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,original_description,amount
0,CARIBOU COFFEE CO,-4.66
1,KRISPY KREME,-89.78
2,CMSVEND,-1.35
3,CMSVEND,-1.35
4,PBC- DENVER CO3834 081023,-2.10
5,USAA CREDIT CARD PAYMENT,-7127.51
6,CANTEEN VENDING MALVERN PA,-2.90
7,XCEL ENERGY-PSCO XCELENERGY,-263.83
8,charge.prezi.com 415-4948313 CA,-60.00
9,203 TAKE 5 CAR WASH BROOMFIELD CO,-39.99


In [150]:
debit_orig_description_totals_df = sum_orig_desc_col(debit_orig_description_amounts_df)
debit_orig_description_totals_df[:50]

,original_description,amount
0,203 TAKE 5 CAR WASH BROOMFIELD CO,-319.92
1,224PEPSIVEN9147678600 DENVER CO,-101.25
2,7-ELEVEN BROOMFIELD CO,-83.36
3,7-ELEVEN 33481 GOLDEN CO,-12.39
4,7-ELEVEN7285 W 119TH PLACBROOMFIELD CO,-74.25
5,7ELEVEN-FCTI11275 MAIN STBROOMFIELD CO,-80.00
6,AAA ACG MBR 006/DUES-AR 3037538800 CO,-185.00
7,ADOBE,-20.80
8,ADOBE,-9.99
9,AGJEANS.COM,-411.84


In [151]:
debit_orig_description_totals_df[50:100]

,original_description,amount
50,DD DOORDASH GOODTIMES,-33.53
51,DD DOORDASH IHOP,-226.18
52,DD DOORDASH JASONSDEL,-145.54
53,DD DOORDASH KNEADERSB,-66.01
54,DD DOORDASH MIKASUSHI,-546.38
55,DD DOORDASH NOODLESCO,-195.73
56,DD DOORDASH OKINAWASU,-115.64
57,DD DOORDASH PANDAEXPR,-60.66
58,DD DOORDASH TCBY,-39.31
59,DICK'S CLOTHING&SPORTING BROOMFIELD CO,-275.09


In [154]:
debit_orig_description_totals_df[100:]

,original_description,amount
100,Nintendo CA948290101 800-2553700 WA,-9.19
101,PANDA EXPRESS,-30.15
102,PANDA EXPRESS 544 WESTMINSTER CO,-34.94
103,PARCHMENT-UNIV DOCS,-37.95
104,PBC- DENVER CO3834 081023,-2.10
105,RISE TV HTTPSRISE.TV FL,-108.00
106,ROAD RUNNER SPORTS,-49.99
107,ROKFIN 512-3876531 TX,-104.93
108,S & S,-31.52
109,SEI 38135 BROOMFIELD CO,-94.57


## Credit Card Analysis

In [160]:
cc_df = pd.read_csv("/content/cc_transactions_Jan-Aug_2023.csv")
cc_df

,Date,Description,Original Description,Category,Amount,Status
0,2023-08-11,TARGET.COM,TARGET.COM,Category Pending,-97.15,Pending
1,2023-08-11,KING SOOPERS #5086,KING SOOPERS #5086,Category Pending,-153.14,Pending
2,2023-08-11,WALMART.COM 8009666546,WALMART.COM 8009666546,Category Pending,-245.42,Pending
3,2023-08-11,USAA CREDIT CARD PAYMENT,USAA CREDIT CARD PAYMENT,Category Pending,7127.51,Pending
4,2023-08-08,AMAZON.COM AMZN.COM/BI,AMAZON.COM AMZN.COM/BI,Category Pending,-107.34,Pending
...,...,...,...,...,...,...
495,2023-03-13,PetSmart,PETSMART # 1458 WESTMINSTER CO,Pet Food & Supplies,-44.32,Posted
496,2023-03-13,USAA Credit Card,USAA CREDIT CARD PAYMENT SAN ANTONIO TX,Transfer,3000.00,Posted
497,2023-03-12,King Soopers,KING SOOPERS #5086 303-439-0113 CO,Groceries,-183.20,Posted
498,2023-03-12,Prime Video,Prime Video*HG9V41WS0 888-802-3080 WA,Television,-5.40,Posted


In [161]:
cc_df = refactor_debit_cols(cc_df)
cc_df

,date,description,original_description,category,amount
0,2023-08-11,TARGET.COM,TARGET.COM,Category Pending,-97.15
1,2023-08-11,KING SOOPERS #5086,KING SOOPERS #5086,Category Pending,-153.14
2,2023-08-11,WALMART.COM 8009666546,WALMART.COM 8009666546,Category Pending,-245.42
3,2023-08-11,USAA CREDIT CARD PAYMENT,USAA CREDIT CARD PAYMENT,Category Pending,7127.51
4,2023-08-08,AMAZON.COM AMZN.COM/BI,AMAZON.COM AMZN.COM/BI,Category Pending,-107.34
...,...,...,...,...,...
495,2023-03-13,PetSmart,PETSMART # 1458 WESTMINSTER CO,Pet Food & Supplies,-44.32
496,2023-03-13,USAA Credit Card,USAA CREDIT CARD PAYMENT SAN ANTONIO TX,Transfer,3000.00
497,2023-03-12,King Soopers,KING SOOPERS #5086 303-439-0113 CO,Groceries,-183.20
498,2023-03-12,Prime Video,Prime Video*HG9V41WS0 888-802-3080 WA,Television,-5.40


In [162]:
cc_orig_desc_amounts_df = clean_orig_desc_col(cc_df)
cc_orig_desc_amounts_df[:50]

<ipython-input-144-a64b9d33a88e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('*')[0])
<ipython-input-144-a64b9d33a88e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('#')[0])
<ipython-input-144-a64b9d33a88e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,original_description,amount
0,TARGET.COM,-97.15
1,KING SOOPERS,-153.14
2,WALMART.COM 8009666546,-245.42
3,USAA CREDIT CARD PAYMENT,7127.51
4,AMAZON.COM AMZN.COM/BI,-107.34
5,COBBLESTONE CAR WASH 72 602-7870808 CO,-37.00
6,PHILLIPS 66 - SEI 38135 BROOMFIELD CO,-43.27
7,AMZN Mktp US,-70.74
8,NINJA NATION CENTENNIAL ENGLEWOOD CO,-17.50
9,AMZN Mktp US,-11.33


In [163]:
cc_orig_description_totals_df = sum_orig_desc_col(cc_orig_desc_amounts_df)
cc_orig_description_totals_df[:50]

,original_description,amount
0,1594 VILLA ITALIAN KIT 303-4691737 CO,-89.30
1,ABLE KIDS FOUNDATION 9702262253 CO,-261.96
2,AFW - E-STORE,-988.45
3,AMAZON.COM,-307.34
4,AMAZON.COM AMZN.COM/BI,-107.34
5,AMAZON.COM AMZN.COM/BILL AMZN.COM CREDIT,122.20
6,AMC 0092 W.MINSTER PRO WESTMINSTER CO,-32.35
7,AMC ONLINE 9640,-32.19
8,AMZN MKTP US,-206.11
9,AMZN MKTP US AMZN.COM/BIL AMZN.CO CREDIT,68.61


In [172]:
cc_cat_df = sum_category_col(cc_df)
cc_cat_df

,category,amount
0,Atm Fee,20.95
1,Auto & Transport,-289.32
2,Bonus,998.70
3,Books,-147.55
4,Business Services,-47.00
5,Cash,-1230.95
6,Category Pending,-7226.75
7,Check,-3382.13
8,Clothing,-2559.32
9,Coffee Shops,-258.31


In [174]:
cc_desc_df = sum_desc_col(cc_df)
cc_desc_df[:50]

,description,amount
0,AMAZON.COM AMZN.COM/BI,-107.34
1,AMC Theatres,-64.54
2,Able Kids Foundation,-261.96
3,Afw E Store,-988.45
4,Amazon,-2597.53
5,Amazon Kids+,-16.20
6,Amznfreetime,-10.80
7,Arvada Pediatric Assoc,-633.43
8,Atgpay Greenway Park,-655.83
9,Banfield,-268.82


In [175]:
cc_desc_df[50:]

,description,amount
50,Party City,-223.48
51,Pediatric Speech Therapy,-2358.00
52,PetSmart,-515.05
53,Phillips 66,-327.23
54,Pizza Hut,-510.81
55,Pretty Nails Inc,-78.00
56,Prime Video,-230.53
57,Raising Cane's,-125.22
58,Rocky Mountain Gifts,-54.31
59,Rocky Mountain Majest,-4.35
